##**DeepFake GAN Project**

#Padrick Beggs

This project creates a generated image of a persons face using  a pre-trained model from the stylegan repository (cited below). Then the project will use this image as the source image for a Deepfake and a video of me talking as the driving video. This part of the project is done by implementing the first-order-model repository (also cited below).

Link to the stylegan repository:

https://github.com/NVlabs/stylegan


Link to the first-order-model:

https://github.com/AliaksandrSiarohin/first-order-model



Both repositories are coding using tensorflow 1. So we must revert back to an older version.

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


This mounts the notebook to google drive. After you run this cell you must click the link to the new URL to get the authorization code. 

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Clones both repositories from github: the stylegan and first-order-model.

In [3]:
#StyleGAN
!git clone https://github.com/NVlabs/stylegan.git


#DeepFake
!git clone https://github.com/AliaksandrSiarohin/first-order-model.git


Cloning into 'stylegan'...
remote: Enumerating objects: 83, done.
remote: Total 83 (delta 0), reused 0 (delta 0), pack-reused 83
Unpacking objects: 100% (83/83), done.
Cloning into 'first-order-model'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 212 (delta 5), reused 11 (delta 3), pack-reused 197
Receiving objects: 100% (212/212), 71.45 MiB | 41.45 MiB/s, done.
Resolving deltas: 100% (102/102), done.


#**styleGAN implementation**

This will create the image we will use as the source image in the Deepfake.

In [4]:
#This changes the current working directory to the stylegan

%cd stylegan/
!ls /content/stylegan/

/content/stylegan
config.py	     LICENSE.txt	    run_metrics.py
dataset_tool.py      metrics		    stylegan-teaser.png
dnnlib		     pretrained_example.py  training
generate_figures.py  README.md		    train.py


This creates a path to a new folder for the stylegan.

In [0]:


import sys
sys.path.insert(0, "/content/stylegan")

These are all the import statements needed to make the stylegan function.

In [6]:
import pickle
import os
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config

The following loads the pretrained weights for the stylegan. 

I created my own link from another drive account because their link is not avalible. After the .com/ you must change "open?" to "uc?" for future reference if this link is also down one day. 

In [7]:
    #This initializes tensorflow
    tflib.init_tf()

    # This loads the karras2019stylegan-ffhq-1024x1024.pkl. This is a pickle file with a pre-trained network.
    # _G is the pickle file with the generator in it.
    # _D is the pickle file with the discriminator in it.
    # Gs is the pickle file with the average of the generator in it. It is cited to give better results.

    with dnnlib.util.open_url('https://drive.google.com/uc?id=1841b_PGZa5Ns0Nhb9kyWdC9SNOyeRBpb', cache_dir=config.cache_dir) as f:
      
        _G, _D, Gs = pickle.load(f)
       











Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


This cell generates the GAN of a fake person's face. It will print "ready" when it is ready. 

In [47]:


    # Picks a latent vector with a random state.
    rnd = np.random.RandomState()
    latents = rnd.randn(1, Gs.input_shape[1])

    #Generates a GAN image
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)

    #Saves the generated image 
    os.makedirs(config.result_dir, exist_ok=True)
    png_filename = os.path.join(config.result_dir, f'/content/drive/My Drive/Colab_Notebooks/DeepFakeGANProject/GANforDeepFake.png')
    PIL.Image.fromarray(images[0], 'RGB').save(png_filename)


    #Prints ready when the stylegan is saved and ready to be implemented.
    print("Ready")


Ready


#**DeepFake implementation**

This will be the second and final part of the project. It will use the stylegan image and the source image for the implementation of the first-order-model.

These are all the import statements needed to make the first-order-model function.

In [0]:
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

This cell instantiates the source image and the driving video using imagio.imread to copy the video and picture from thier respective paths. Then, after resizing the images, it displays both of them side by side. 

In [52]:
source_image = imageio.imread('/content/drive/My Drive/Colab_Notebooks/DeepFakeGANProject/GANforDeepFake.png')
driving_video = imageio.mimread('/content/drive/My Drive/Colab_Notebooks/DeepFakeGANProject/drivingVideo.mp4', memtest=False)


  # This resizes the source image and the driving video to be 256 by 256. 
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]


  # Defines the function that displays a video and an image side by side. In our case, the driving video and the source image. 
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

  # Displays the driving video and source image. 
HTML(display(source_image, driving_video).to_html5_video())



This creates a path to a new folder for the first-order-model.

In [0]:

sys.path.insert(0, "/content/first-order-model")

from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='/content/first-order-model/config/vox-256.yaml', 
                            checkpoint_path='/content/drive/My Drive/Colab_Notebooks/DeepFakeGANProject/first-order-motion-model/vox-cpk.pth.tar')

from demo import make_animation
from skimage import img_as_ubyte


**This cell creates the final product of this project. A fake video of a equally fake person.**

After it is generated it is saved as an mp4 file and can be downloaded from the content folder. After it is saved it is displayed along with the source image and the driving video. 

In [49]:

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())


100%|██████████| 207/207 [00:07<00:00, 27.16it/s]


##**Citations**

This project would have not been possible without the following resources. 

Jeff Heaton:

https://www.youtube.com/watch?v=RPGOPrkieTE

Aliaksandr Siarohin:

https://colab.research.google.com/github/AliaksandrSiarohin/first-order-model/blob/master/demo.ipynb#scrollTo=rW-ipQXPOWUo

https://github.com/AliaksandrSiarohin/first-order-model

NVLabs:

https://github.com/NVlabs/stylegan

